Договоренности о функциях потерь:
1. Для регрессии reg: linear
2. Двоичная классифкация reg: logistic - выход категория целевого объекта; binary: logistic - вероятность положительного класса  

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
class_data = pd.read_csv("classification_data.csv")
X, y = class_data.iloc[:,:-1], class_data.iloc[:,-1]
X_train, X_test, y_train, y_test= train_test_split(X, y,
test_size=0.2, random_state=123)
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)
xg_cl.fit(X_train, y_train)
preds = xg_cl.predict(X_test)
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy)) #accuracy: 0.78333


In [ ]:
#Cross-validation in XGBoost example
import xgboost as xgb
import pandas as pd
churn_data = pd.read_csv("classification_data.csv")
churn_dmatrix = xgb.DMatrix(data=churn_data.iloc[:,:-1], label=churn_data.month_5_still_here) # data = X, label = y
params={"objective":"binary:logistic","max_depth":4}
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, nfold=4, num_boost_round=10, metrics="error", as_pandas=True)
#данные, параметры, кол-во фолдов в cross-validation, кол-во деревьев, "error" - преобразуется в accuracy через 1 - "_error-mean"
print("Accuracy: %f" %((1-cv_results["test-error-mean"]).iloc[-1]))
# у такого подхода есть cross-validation оценка

In [ ]:
#регрессионная модель с деревьями с помощью API совместимого с scikit-learn
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
boston_data = pd.read_csv("boston_housing.csv")
X, y = boston_data.iloc[:,:-1],boston_data.iloc[:,-1]
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)
xg_reg = xgb.XGBRegressor(objective='reg:linear', n_estimators=10, seed=123)
xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f" % (rmse))

In [ ]:
#использование только API XGBoost
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
boston_data = pd.read_csv("boston_housing.csv")
X, y = boston_data.iloc[:,:-1],boston_data.iloc[:,-1]

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)
DM_train = xgb.DMatrix(data=X_train,label=y_train) #преобразование набора в объекты DMatrix
DM_test = xgb.DMatrix(data=X_test,label=y_test)
params = {"booster":"gblinear", "objective":"reg:linear"} #создаем словарь параметров, явно указывающий базового учащегося
#"booster":"gblinear" - linear learner
xg_reg = xgb.train(params = params, dtrain=DM_train, num_boost_round=10)
preds = xg_reg.predict(DM_test)
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f" % (rmse))

Параметры регуляризации в XGBoost
1. gamma (для деревьев) - чем больше тем меньше разделений  
2. alpha - L1 регуляризация для весов листьев, многие веса идут к нулю
3. lambda - L2 для листев

In [ ]:
import xgboost as xgb
import pandas as pd
boston_data = pd.read_csv("boston_data.csv")
X,y = boston_data.iloc[:,:-1],boston_data.iloc[:,-1]
boston_dmatrix = xgb.DMatrix(data=X,label=y)
params={"objective":"reg:linear", "max_depth":4}
l1_params = [1,10,100]
rmses_l1=[]
for reg in l1_params:
  params["alpha"] = reg
  cv_results = xgb.cv(dtrain=boston_dmatrix, params=params, nfold=4,
  num_boost_round=10, metrics="rmse", as_pandas=True, seed=123)
  rmses_l1.append(cv_results["test-rmse-mean"].tail(1).values[0])
print("Best rmse as a function of l1:")
print(pd.DataFrame(list(zip(l1_params,rmses_l1)), columns=["l1", "rmse"]))

In [ ]:
# отрисовка ка каждого дерева
xg_reg = xgb.train(params=params, dtrain=housing_dmatrix, num_boost_round=10)

# Plot the first tree
xgb.plot_tree(xg_reg, num_trees=0)
plt.show()

# Plot the fifth tree
xgb.plot_tree(xg_reg, num_trees=1)
plt.show()

# Plot the last tree sideways
xgb.plot_tree(xg_reg, num_trees=2)
plt.show()

In [ ]:
# гистограмма важности признака (для tree) - кол-во раз когда признак был в разбиении
housing_dmatrix = xgb.DMatrix(data=X, label=y)
params = {"objective":"reg:linear", "max_depth":4}
xg_reg = xgb.train(params=params, dtrain=housing_dmatrix, num_boost_round=10)

# Plot the feature importances
xgb.plot_importance(xg_reg)
plt.show()

In [ ]:
# ранняя остновка
housing_dmatrix = xgb.DMatrix(data=X, label=y)
params = {"objective":"reg:linear", "max_depth":4}
# Perform cross-validation with early stopping: cv_results
cv_results = xgb.cv(dtrain = housing_dmatrix, params = params, seed = 123, metrics = 'rmse', early_stopping_rounds = 10,  num_boost_round=50, nfold = 3, as_pandas = True)
# Print cv_results
print(cv_results)

При более высоких значениях "eta" (learning rate) штрафные веса признаков становятся более значительными, что приводит к гораздо более сильной регуляризации.

colsample_bytree - от 0 до 1 доля признаков при каждом разбиении

In [ ]:
housing_dmatrix = xgb.DMatrix(data=X, label=y)
params = {"objective":"reg:linear", "max_depth":3}

eta_vals = [0.001, 0.01, 0.1]
best_rmse = []

for curr_val in eta_vals:
    params["eta"] = curr_val
    cv_results = xgb.cv(seed = 123, metrics = 'rmse', params = params, dtrain = housing_dmatrix, early_stopping_rounds = 5,  num_boost_round=10, nfold = 3, as_pandas = True)
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

print(pd.DataFrame(list(zip(eta_vals, best_rmse)), columns=["eta","best_rmse"]))

In [ ]:
#лучшие модификации для colsample_bytree, learning_rate и max_depth
import pandas as pd
import xgboost as xgb
import numpy as np
housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
X,y = housing_data[housing_data.columns.tolist()[:-1]], housing_data[housing_data.columns.tolist()[-1]]
housing_dmatrix = xgb.DMatrix(data=X,label=y)
untuned_params={"objective":"reg:linear"}
untuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix, params=untuned_params,nfold=4, metrics="rmse", as_pandas=True,seed=123)
print("Untuned rmse: %f" %((untuned_cv_results_rmse["test-rmse-mean"]).tail(1)))

In [ ]:
#еще лучше количество деревьев 200
import pandas as pd
import xgboost as xgb
import numpy as np
housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
X,y = housing_data[housing_data.columns.tolist()[:-1]], housing_data[housing_data.columns.tolist()[-1]]
housing_dmatrix = xgb.DMatrix(data=X,label=y)
tuned_params = {"objective":"reg:linear", 'colsample_bytree': 0.3, 'learning_rate': 0.1,'max_depth': 5}
tuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix, params=tuned_params, nfold=4, num_boost_round=200, metrics="rmse", as_pandas=True, seed=123)
print("Tuned rmse: %f" %((tuned_cv_results_rmse["test-rmse-mean"]).tail(1)))

Настраиваемые параметры будут отличаться в зависимости от базового алгоритма
Для дерева:
1. learning rate
2. gamma
3. lambda
4. alpha
5. max_depth
6. subsample
7. colsample_bytree

Для линейного базового алгоритма:
1. lambda
2. alpha  
3. lambda_bias

In [ ]:
#поиск по сетке
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV
housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
X, y = housing_data[housing_data.columns.tolist()[:-1]], housing_data[housing_data.columns.tolist()[-1]]
housing_dmatrix = xgb.DMatrix(data=X,label=y) # зачем
gbm_param_grid = {'learning_rate': [0.01,0.1,0.5,0.9], 'n_estimators': [200], 'subsample': [0.3, 0.5, 0.9]}
gbm = xgb.XGBRegressor()
grid_mse = GridSearchCV(estimator=gbm,param_grid=gbm_param_grid, scoring='neg_mean_squared_error', cv=4, verbose=1)
grid_mse.fit(X, y)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
#случайный поиск
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
X,y = housing_data[housing_data.columns.tolist()[:-1]],
housing_data[housing_data.columns.tolist()[-1]]
housing_dmatrix = xgb.DMatrix(data=X,label=y) # зачем
gbm_param_grid = {'learning_rate': np.arange(0.05,1.05,.05), 'n_estimators': [200], 'subsample': np.arange(0.05,1.05,.05)}
gbm = xgb.XGBRegressor()
randomized_mse = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid, n_iter=25, scoring='neg_mean_squared_error', cv=4, verbose=1)
randomized_mse.fit(X, y) # param_distributions - отличие
print("Best parameters found: ",randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

In [ ]:
#делаем pipeline для случайного леса
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
names = ["crime","zone","industry","charles","no","rooms","age","distance","radial","tax","pupil","aam","lower","med_price"]
data = pd.read_csv("boston_housing.csv", names=names)
X, y = data.iloc[:,:-1], data.iloc[:,-1]
rf_pipeline = Pipeline[("st_scaler", StandardScaler()), ("rf_model", RandomForestRegressor())]
scores = cross_val_score(rf_pipeline,X,y, scoring="neg_mean_squared_error", cv=10)
final_avg_rmse = np.mean(np.sqrt(np.abs(scores)))
print("Final RMSE:", final_avg_rmse)

In [ ]:
#делаем pipeline для XGBoost через Scikit-learn API
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
names = ["crime","zone","industry","charles","no","rooms","age","distance","radial","tax","pupil","aam","lower","med_price"]
data = pd.read_csv("boston_housing.csv",names=names)
X, y = data.iloc[:,:-1], data.iloc[:,-1]
xgb_pipeline = Pipeline[("st_scaler", StandardScaler()), ("xgb_model",xgb.XGBRegressor())]
scores = cross_val_score(xgb_pipeline, X, y, scoring="neg_mean_squared_error",cv=10)
final_avg_rmse = np.mean(np.sqrt(np.abs(scores)))
print("Final XGB RMSE:", final_avg_rmse)

In [ ]:
# практика с векторизацией
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor(max_depth=2, objective="reg:linear"))]
xgb_pipeline = Pipeline(steps)
cross_val_scores = cross_val_score(xgb_pipeline, X.to_dict('records'), y, scoring = 'neg_mean_squared_error', cv = 10)
print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))

In [ ]:
#Настройка гиперпараметров xgboost в pipeline
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
names = ["crime","zone","industry","charles","no","rooms","age","distance","radial","tax","pupil","aam","lower","med_price"]
data = pd.read_csv("boston_housing.csv", names=names)
X, y = data.iloc[:,:-1],data.iloc[:,-1]
xgb_pipeline = Pipeline[("st_scaler", StandardScaler()), ("xgb_model", xgb.XGBRegressor())]
gbm_param_grid = {'xgb_model__subsample': np.arange(.05, 1, .05), 'xgb_model__max_depth': np.arange(3,20,1), 'xgb_model__colsample_bytree': np.arange(.1,1.05,.05) }
randomized_neg_mse = RandomizedSearchCV(estimator=xgb_pipeline, param_distributions=gbm_param_grid, n_iter=10, scoring='neg_mean_squared_error', cv=4)
randomized_neg_mse.fit(X, y)
print("Best rmse: ", np.sqrt(np.abs(randomized_neg_mse.best_score_)))
print("Best model: ", randomized_neg_mse.best_estimator_)

Преобразование данных

In [ ]:
from sklearn.preprocessing import LabelEncoder
df.LotFrontage = df.LotFrontage.fillna(0) # заполение NaN нулями
categorical_mask = (df.dtypes == 'object') # создание маски где тип object

categorical_columns = df.columns[categorical_mask].tolist() # получаем список категорильных столбцов
print(df[categorical_columns].head())

le = LabelEncoder() # три катеогории - 0, 1, 2
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))
print(df[categorical_columns].head())

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
df_encoded = ohe.fit_transform(df)

#или все проще
from sklearn.feature_extraction import DictVectorizer
df_dict = df.to_dict('records')
dv = DictVectorizer(sparse=False)
df_encoded = dv.fit_transform(df_dict)
print(dv.vocabulary_)

#суем в пайплайн
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

X.LotFrontage = X.LotFrontage.fillna(0)
steps = [("ohe_onestep", DictVectorizer(sparse=False)), ("xgb_model", xgb.XGBRegressor())]
xgb_pipeline = Pipeline(steps)
xgb_pipeline.fit(X.to_dict("records"), y)

Преобразование признаков c sklearn_pandas (конспект)

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.impute import SimpleImputer

nulls_per_column = X.isnull().sum()
print(nulls_per_column)

categorical_feature_mask = X.dtypes == object
categorical_columns = X.columns[categorical_feature_mask].tolist()
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

numeric_imputation_mapper = DataFrameMapper([([numeric_feature], SimpleImputer(strategy="median")) for numeric_feature in non_categorical_columns],
                                            input_df=True, df_out=True)
categorical_imputation_mapper = DataFrameMapper([(category_feature, SimpleImputer()) for category_feature in categorical_columns],
                                                input_df=True, df_out=True)

from sklearn.pipeline import FeatureUnion
numeric_categorical_union = FeatureUnion([("num_mapper", numeric_imputation_mapper), ("cat_mapper", categorical_imputation_mapper)])

pipeline = Pipeline([("featureunion", numeric_categorical_union),
                     ("dictifier", Dictifier()),
                     ("vectorizer", DictVectorizer(sort=False)),
                     ("clf", xgb.XGBClassifier(max_depth=3))])

cross_val_scores = cross_val_score(pipeline, X, y, scoring="roc_auc", cv=3)
print("3-fold AUC: ", np.mean(cross_val_scores))

gbm_param_grid = {
    'clf__learning_rate': np.arange(0.05, 1, 0.05),
    'clf__max_depth': np.arange(3, 10, 1),
    'clf__n_estimators': np.arange(50, 200, 50)}

randomized_roc_auc = RandomizedSearchCV(estimator=pipeline, param_distributions=gbm_param_grid, n_iter=2, scoring='roc_auc', cv=2, verbose=1)

randomized_roc_auc.fit(X, y)

print(randomized_roc_auc.best_score_)
print(randomized_roc_auc.best_estimator_)